# SCIDONI : Prediction of the isotopic inventory in a nuclear reactor core

*Benjamin Dechenaux & Jean-Baptiste Clavel (IRSN, PSN-EXP/SNC/LN)*

## Introduction 

This project aims at the prediction of the time evolving composition of the nuclear fuel inside a reactor.

Building a prediction model capable of predicting such inventory is usually done through computer expensive "Monte-Carlo" calculations and the goal here would be to build a fast and relatively precise model to use in an emergency situation such as the occurence of a severe accident.


In [ ]:
# Required for running the notebook 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib nbagg 
import seaborn as sns 
import pickle

## About the data 

TO BE DONE 

### Description of the available data

TO BE DONE 

The 1120 simulations have been splitted into two *training* and *testing* datasets.

* The *training* dataset is composed of 920 and are accessible in CSV format undert the __train__ folder
* The *testing* dataset is composed of 200 and are accessible in CSV format undert the __test__ folder


### Loading the data 

To ease the reading of both the training and testing datasets, serialized *pandas* dataframes have been pre-prepared

In [3]:
dtrain = pickle.load( open("data/train_data_python3.pickle", "rb") )
dtest = pickle.load( open("data/test_data_python3.pickle", "rb") )

Here , data are concatenated each on top of the others , etc....

TO BE DONE 


In [4]:
dtrain.head(5)

,times,A,B,C,D,E,F,G,H,I,...,V,W,X,Y,Z,p1,p2,p3,p4,p5
0,0.0,0.173951,4.219378,0.088625,2.014683,0.677343,0.079891,0.054258,0.164877,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.04842,0.029876,0.043997,0.03811,0.042866
1,20.0,0.172232,4.214907,0.088131,1.995882,0.678855,0.083688,0.054383,0.162316,0.000005,...,0.000038,0.000005,0.000160,0.000158,0.000362,0.04842,0.029876,0.043997,0.03811,0.042866
2,40.0,0.170516,4.210405,0.087780,1.977858,0.680277,0.087465,0.054536,0.159785,0.000031,...,0.000084,0.000022,0.000540,0.000316,0.000485,0.04842,0.029876,0.043997,0.03811,0.042866
3,60.0,0.168814,4.205899,0.087564,1.959920,0.681660,0.091179,0.054690,0.157300,0.000092,...,0.000132,0.000050,0.000999,0.000474,0.000526,0.04842,0.029876,0.043997,0.03811,0.042866
4,80.0,0.167123,4.201368,0.087465,1.942086,0.682998,0.094833,0.054855,0.154862,0.000190,...,0.000182,0.000088,0.001487,0.000631,0.000540,0.04842,0.029876,0.043997,0.03811,0.042866


In [16]:
dtrain.loc[0].shape  # equivalent to dtrain.loc[dtrain["times"] == 0.]


(920, 32)